In [7]:
import argparse
import os
import re
import sys
import warnings
from itertools import product
from pathlib import Path
from typing import Any

import joblib
import numpy as np  # type: ignore
import pandas as pd
import xarray as xr  # type: ignore
from rich.progress import track
from keras import Model

from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [2]:
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "5"  # hide stuff
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"] = "true"  # should fix most memory issues

In [3]:
import keras
import tensorflow as tf

warnings.simplefilter("ignore")

In [4]:
to_drop = "reflectivity"

CNN_FEATURES = [
    "cloud_flag",
    "temp",
    "mpl_backscatter",
    "mpl_linear_depol_ratio",
    "reflectivity",
    "radar_linear_depolarization_ratio",
    "spectral_width",
    "mean_doppler_velocity",
    "mwrret1liljclou_be_lwp",
]

In [8]:
def add_argmax(model: Model) -> Model:
    """Adds an argmax layer to the model to ensure outputs are 0-7 instead of an array
    with length 7. This makes it easier to compare with the y_test labels."""
    argmax_layer = keras.layers.Lambda(lambda x: tf.argmax(x, axis=-1), name='argmax_layer')
    return Model(inputs=model.input, outputs=argmax_layer(model.output))

cnn: Model = add_argmax(keras.models.load_model("../processing/models/cnn.20240429.213223.h5"))  # type: ignore


In [22]:
X_test = np.load("../preprocessing/data/cnn_inputs/X_test.npy")
y_test = np.squeeze(np.load("../preprocessing/data/cnn_inputs/y_test.npy"))  # (S, 128, 384, 1) --> (S, 128, 384)
y_flat = y_test.ravel()  # flatten to 1D array (S x 128 x 2384)
print(f"{X_test.shape = }")
print(f"{y_test.shape = }")

X_test.shape = (6096, 128, 384, 9)
y_test.shape = (6096, 128, 384)


In [27]:
S = 500
X, y = X_test[:S], y_test[:S].ravel()
X.shape

(500, 128, 384, 9)

In [28]:
pred = cnn.predict(X).ravel()

16/16 [==============================] - 31s 2s/step


In [29]:
def acc(y_pred, y_true) -> float:
    cloudy = np.where(~np.isin(y_true, [0, 8]))
    return (y_pred[cloudy] == y_true[cloudy]).mean()

acc(pred, y)

0.9140456029715127

In [30]:
X_missing = np.copy(X)
X_missing[:, :, :, CNN_FEATURES.index("reflectivity")] = 0

pred_missing = cnn.predict(X_missing, verbose=0).ravel()
acc(pred_missing, y)


0.8581331014433918

In [31]:
pred_missing2 = cnn.predict(X_missing / (1-0.125), verbose=0).ravel()
acc(pred_missing2, y)

0.8570286256445979